In [1]:
import numpy as np
from random import random

In [5]:
sizes = [50, 500, 1000, 2000, 5000, 10000, 50000, 100000, 1000000, 10000000]

In [6]:
def write(sequence, name):
    with open(name, 'w') as out:
        for val in sequence:
            out.write("{}\n".format(val))    

def gen_rand(size, name):
    data = np.arange(size)
    np.random.shuffle(data)
    write(data, name)

def gen_rev(size, name):
    write(np.arange(size)[::-1], name)
    
def gen_order(size, name):
    write(np.arange(size), name)

In [7]:
for size in sizes:
    gen_rand(size, "../input/random_{}.txt".format(size))

    gen_rev(size, "../input/reverse_{}.txt".format(size))
    
    gen_order(size, "../input/order_{}.txt".format(size))